In [5]:
### This is a try with the former dataset using undersampling
from imblearn.under_sampling import *
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import os
import numpy as np
from pathlib import Path

In [8]:
path = Path("D:/fast_ai/NLP/cap")

In [13]:
df1 = pd.read_pickle(path/"df_1.pkl")


In [24]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split, cross_val_score
from sklearn.metrics import fbeta_score, make_scorer, confusion_matrix, balanced_accuracy_score, accuracy_score,classification_report
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler


In [17]:

vectorizer = TfidfVectorizer(min_df=2,max_df=0.95,ngram_range=(1,5),stop_words="english")


features = vectorizer.fit_transform(df1["reviews.text"])




In [19]:

x_train, X_test,y_train,Y_test= train_test_split(features,df1["reviews.rating"],test_size=0.25,random_state=123)
y_train.value_counts(normalize=True),Y_test.value_counts(normalize=True)

(5    0.703361
 4    0.187142
 3    0.043887
 1    0.040144
 2    0.025466
 Name: reviews.rating, dtype: float64, 5    0.704756
 4    0.182739
 3    0.045795
 1    0.042492
 2    0.024218
 Name: reviews.rating, dtype: float64)

In [20]:
### slightly undersample training set's dominant target 
rus = NearMiss(sampling_strategy={5:3500})
x_train ,y_train = rus.fit_resample(x_train,y_train)
pd.value_counts(y_train)

5    3500
4    2550
3     598
1     547
2     347
Name: reviews.rating, dtype: int64

In [21]:
### make SGD Pipeline
clf = SGDClassifier(random_state=22,class_weight="balanced",n_jobs=-1)

parameters = [{ 'loss': ['hinge', 'log', 'perceptron'], 
                'alpha': 10.0**-np.arange(1,7),
                'penalty': ['l1', 'l2', 'elasticnet']}]
clf_SGD_refined = GridSearchCV(SGDClassifier(random_state=22,n_jobs=-1,class_weight="balanced"), parameters)
clf_SGD_refined.fit(x_train,y_train)


GridSearchCV(cv=None, error_score=nan,
             estimator=SGDClassifier(alpha=0.0001, average=False,
                                     class_weight='balanced',
                                     early_stopping=False, epsilon=0.1,
                                     eta0=0.0, fit_intercept=True,
                                     l1_ratio=0.15, learning_rate='optimal',
                                     loss='hinge', max_iter=1000,
                                     n_iter_no_change=5, n_jobs=-1,
                                     penalty='l2', power_t=0.5, random_state=22,
                                     shuffle=True, tol=0.001,
                                     validation_fraction=0.1, verbose=0,
                                     warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid=[{'alpha': array([1.e-01, 1.e-02, 1.e-03, 1.e-04, 1.e-05, 1.e-06]),
                          'loss': ['hinge', 'log', 'perceptron'],
         

In [25]:
preds = clf_SGD_refined.predict(X_test)
print(classification_report(Y_test,preds,digits=4))

              precision    recall  f1-score   support

           1     0.4913    0.5855    0.5343       193
           2     0.1944    0.1273    0.1538       110
           3     0.2748    0.1731    0.2124       208
           4     0.2585    0.6831    0.3751       830
           5     0.8366    0.5008    0.6265      3201

    accuracy                         0.5137      4542
   macro avg     0.4111    0.4140    0.3804      4542
weighted avg     0.6750    0.5137    0.5463      4542

